In [44]:
import numpy as np
import time
import random

# Define the game board representation
def create_board():
    """Create an empty 3x3 Tic-Tac-Toe board."""
    return np.array([[' ' for _ in range(3)] for _ in range(3)])

# Print the current state of the board
def print_board(board):
    """Display the board in a user-friendly format with clearer formatting."""
    print("\n  " + "   ".join(["0", "1", "2"]))  # Column indices with more spacing
    print("  " + "+---+---+---+")
    
    for i, row in enumerate(board):
        # Replace '_' with space for better visibility
        formatted_row = [cell if cell != '_' else ' ' for cell in row]
        print(f"{i} | {' | '.join(formatted_row)} |")
        print("  " + "+---+---+---+")

In [45]:
# Check if there are empty spaces on the board
def is_board_full(board):
    """Return True if the board is full, False otherwise."""
    return ' ' not in board.flatten()

# Check if a player has won
def check_winner(board, player):
    """Check if the specified player has won."""
    # Check rows
    for row in range(3):
        if all(board[row, col] == player for col in range(3)):
            return True
    
    # Check columns
    for col in range(3):
        if all(board[row, col] == player for row in range(3)):
            return True
    
    # Check diagonals
    if all(board[i, i] == player for i in range(3)) or all(board[i, 2-i] == player for i in range(3)):
        return True
    
    return False

# Check if the game is over
def is_game_over(board):
    """Check if the game has ended (win or draw)."""
    return check_winner(board, 'X') or check_winner(board, 'O') or is_board_full(board)

# Get a list of available moves (empty cells)
def get_available_moves(board):
    """Return a list of available positions as (row, col) tuples."""
    available_moves = []
    for i in range(3):
        for j in range(3):
            if board[i, j] == ' ':
                available_moves.append((i, j))
    return available_moves

In [46]:
# Function to print the solution steps
def print_solution(solution):
    """Print the steps of a solution."""
    if solution is None:
        print("No solution found.")
        return
    
    print(f"Solution found in {len(solution)} steps:")
    
    for step, (state, action) in enumerate(solution):
        print(f"Step {step + 1}: {action}")
        print(f"State: {state}")
    
    print(f"Total steps: {len(solution)}")

In [47]:
# Implement the Minimax algorithm
def minimax(board, depth, is_maximizing, alpha=float('-inf'), beta=float('inf')):

    # Terminal states
    if check_winner(board, 'O'):  # Computer wins
        return 10 - depth
    if check_winner(board, 'X'):  # Human wins
        return depth - 10
    if is_board_full(board):      # Draw
        return 0
    
    available_moves = get_available_moves(board)
    
    if is_maximizing:  # Computer's turn (O) - maximizing
        max_eval = float('-inf')
        for move in available_moves:
            row, col = move
            board[row, col] = 'O'
            eval_score = minimax(board, depth + 1, False, alpha, beta)
            board[row, col] = ' '  # Undo the move
            max_eval = max(max_eval, eval_score)
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break  # Beta cutoff
        return max_eval
    else:  # Human's turn (X) - minimizing
        min_eval = float('inf')
        for move in available_moves:
            row, col = move
            board[row, col] = 'X'
            eval_score = minimax(board, depth + 1, True, alpha, beta)
            board[row, col] = ' '  # Undo the move
            min_eval = min(min_eval, eval_score)
            beta = min(beta, eval_score)
            if beta <= alpha:
                break  # Alpha cutoff
        return min_eval

In [48]:
# Find the best move for the computer using Minimax
def find_best_move(board):
    """Find the best move for the computer using the Minimax algorithm."""
    best_score = float('-inf')
    best_move = None
    
    available_moves = get_available_moves(board)
    
    for move in available_moves:
        row, col = move
        board[row, col] = 'O'  # Computer's move
        score = minimax(board, 0, False)  # After computer moves, it's human's turn
        board[row, col] = ' '  # Undo the move (using space instead of '_')
        
        if score > best_score:
            best_score = score
            best_move = move
    
    return best_move

# Helper function to get valid move coordinates
def get_valid_input(prompt, valid_range):
    """Get a valid integer input from the user in the specified range."""
    while True:
        try:
            val = input(prompt)
            val = int(val)
            if val in valid_range:
                return val
            else:
                print(f"Value must be in range {min(valid_range)}-{max(valid_range)}")
        except ValueError:
            print("Please enter a valid integer")

# Human player makes a move
def human_move(board):
    """Allow the human player to make a move with clearer UI."""
    print("\n🎮 Your available moves:")
    available_positions = get_available_moves(board)
    position_dict = {i+1: pos for i, pos in enumerate(available_positions)}
    
    # Create a visual representation of available positions
    visual_board = np.copy(board)
    for i, (row, col) in position_dict.items():
        visual_board[row, col] = str(i)
    
    # Display the board with numbered positions
    print("\nBoard with position numbers:")
    print("  " + "   ".join(["0", "1", "2"]))  # Column indices with more spacing
    print("  " + "+---+---+---+")
    
    for i, row in enumerate(visual_board):
        print(f"{i} | {' | '.join(row)} |")
        print("  " + "+---+---+---+")
    
    # Get user input for position
    while True:
        try:
            choice = int(input(f"\n👉 Choose position (1-{len(position_dict)}): "))
            if choice < 1 or choice > len(position_dict):
                print(f"Please enter a number between 1 and {len(position_dict)}")
                continue
            
            # Get the position from the dictionary
            row, col = position_dict[choice]
            break
        except ValueError:
            print("Please enter a valid number")
    
    board[row, col] = 'X'
    print(f"\n✅ You placed X at position ({row}, {col})")

In [49]:
# Play the game
def play_game():
    """Main game loop."""
    board = create_board()
    print("\n" + "="*50)
    print("🎲 WELCOME TO TIC-TAC-TOE WITH MINIMAX! 🎲")
    print("="*50)
    print("\nYou are ❌ (X), and the computer is ⭕ (O).")
    print("\nInitial board:")
    print_board(board)
    
    # Game loop
    while not is_game_over(board):
        # Human's turn
        print("\n" + "*"*30)
        print("👤 YOUR TURN (X)")
        print("*"*30)
        human_move(board)
        print("\nBoard after your move:")
        print_board(board)
        
        # Check if human won or the game is a draw
        if check_winner(board, 'X'):
            print("\n🎉🏆 Congratulations! You won! 🏆🎉")
            return
        if is_board_full(board):
            print("\n🤝 It's a draw! 🤝")
            return
        
        # Computer's turn
        print("\n" + "*"*30)
        print("🤖 COMPUTER'S TURN (O)")
        print("*"*30)
        start_time = time.time()
        row, col = find_best_move(board)
        end_time = time.time()
        
        board[row, col] = 'O'
        print(f"\n⏱️ Computer calculating: {end_time - start_time:.2f} seconds")
        print(f"✓ Computer placed O at position ({row}, {col})")
        print("\nBoard after computer's move:")
        print_board(board)
        
        # Check if computer won or the game is a draw
        if check_winner(board, 'O'):
            print("\n🤖 Computer won! Better luck next time. 🤖")
            return
        if is_board_full(board):
            print("\n🤝 It's a draw! 🤝")
            return

In [50]:
# Start a new game
play_game()


🎲 WELCOME TO TIC-TAC-TOE WITH MINIMAX! 🎲

You are ❌ (X), and the computer is ⭕ (O).

Initial board:

  0   1   2
  +---+---+---+
0 |   |   |   |
  +---+---+---+
1 |   |   |   |
  +---+---+---+
2 |   |   |   |
  +---+---+---+

******************************
👤 YOUR TURN (X)
******************************

🎮 Your available moves:

Board with position numbers:
  0   1   2
  +---+---+---+
0 | 1 | 2 | 3 |
  +---+---+---+
1 | 4 | 5 | 6 |
  +---+---+---+
2 | 7 | 8 | 9 |
  +---+---+---+

✅ You placed X at position (1, 1)

Board after your move:

  0   1   2
  +---+---+---+
0 |   |   |   |
  +---+---+---+
1 |   | X |   |
  +---+---+---+
2 |   |   |   |
  +---+---+---+

******************************
🤖 COMPUTER'S TURN (O)
******************************

⏱️ Computer calculating: 0.20 seconds
✓ Computer placed O at position (0, 0)

Board after computer's move:

  0   1   2
  +---+---+---+
0 | O |   |   |
  +---+---+---+
1 |   | X |   |
  +---+---+---+
2 |   |   |   |
  +---+---+---+

*************